In [0]:
import os
import numpy
#from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D ,GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dense
from keras.layers import Dropout
#from keras.applications.inception_v3 import preprocess_input
from keras.optimizers import Adam
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50

In [0]:
base_model = ResNet50(
    weights='imagenet',
    input_shape=(224,224,3),
    include_top=False,
    pooling='avg')
model = Sequential()
model.add(base_model)
#Here number of labels = 7
model.add(Dense(units = 7, activation='sigmoid'))

In [3]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 23,548,935
Non-trainable params: 53,120
_________________________________________________________________
None


In [0]:
#Mount the google drive
from google.colab import drive
drive.mount("/content/drive")

In [9]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [0]:
train_data_dir = 'PATH/TO/TRAIN/DATASET'
validation_data_dir = 'PATH/TO/VALIDATION/DATASET'

In [0]:
train_datagen = ImageDataGenerator(
    rescale= 1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

In [0]:
#test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical')

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [0]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

W0621 11:56:23.427563 139755546548096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
history= History()
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1,callbacks= [history])

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
model.save("PATH/TO/SAVE/MODEL/ResNet.model")

In [0]:
#EVALUATE the model
model.evaluate_generator(generator=train_generator,steps=STEP_SIZE_TRAIN)

##FINE-TUNING
### *if required*

In [0]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/ResNet/ResNets_10_epoch.model")

model.trainable = True

fine_tune_at = 75

#Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
    layer.trainable =  False


model.compile(optimizer='adam',

               loss='categorical_crossentropy',

               metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 23,548,935
Non-trainable params: 53,120
_________________________________________________________________
None


In [0]:
history= History()
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1,callbacks= [history])